In [1]:
import numpy as np
import matplotlib.pyplot as plt

Matplotlib is building the font cache; this may take a moment.


In [2]:
import scipy.linalg as sl

In [440]:
import afqmctools.systems.lattice as lat
from afqmctools.observables.greens import greens_1body

In [437]:
Nx,Ny = 4,4
N = Nx*Ny
T = np.zeros((N,N))
for i in range(Nx):
    for j in range(Ny):
        a,b = i*Ny+j,i*Ny+(j+1)%Ny
        T[a,b] += -1
        T[b,a] += -1
        a,b = i*Ny+j,((i+1)%Nx)*Ny+j
        T[a,b] += -1
        T[b,a] += -1
assert np.allclose(T,T.conj().T)

In [683]:
lattice = lat.SquareLattice(
            L=(4,4),
            axis1_boundary=lat.PBCBoundary,
            axis2_boundary=lat.PBCBoundary,
            )

In [634]:
lams,vecs = np.linalg.eigh(T)
# filled orbitals, and unfilled (virtual) orbitals
of,ov = vecs[:,:8], vecs[:,8:]

In [635]:
g_up,g_dn = greens_1body(of),greens_1body(of)
np.sum(g_up*T), np.sum(g_dn*T)

(np.float64(-12.0), np.float64(-12.0))

In [636]:
o_up,o_dn = of,of
g_up,g_dn = greens_1body(o_up),greens_1body(o_dn)

U=3.5
np.sum(T*g_up)+np.sum((g_dn)*T)+ U*(g_up.diagonal()@g_dn.diagonal()) 

np.float64(-9.265465844048927)

#### Full Transform

Particle Hole Transformation
$$
c_{i\sigma} \to (-1)^i d^\dagger_{i\sigma} 
$$
$$
\langle c^\dagger_{i\sigma} c_{j\sigma}\rangle \to \langle \delta_{ij} -d^\dagger_{i\sigma} d_{j\sigma}\rangle
$$

This means that $E_K \to E_K$ and $U \to U$ because of Hubbard model PH symmetry


In [637]:
signs_v = (-1)**(np.array([np.sum(s.coord)  for s in lattice.sites]))
signs = np.outer(signs_v,signs_v)


In [638]:
o_up,o_dn = signs_v.reshape(N,1)* ov,signs_v.reshape(N,1)* ov

g_up,g_dn =np.eye(N)-greens_1body(o_up)*signs,np.eye(N)-greens_1body(o_dn)*signs

U=3.5
np.sum(T*(g_up))+np.sum((g_dn)*T)+ U*(g_up.diagonal()@(g_dn.diagonal())) 


np.float64(-9.265465844048931)

#### Partial Transform

Particle Hole Transformation
$$
c_{i\uparrow} \to d_{i\uparrow} 
$$
$$
c_{i\downarrow} \to (-1)^i d^\dagger_{i\downarrow}
$$

This means that $E_K \to E_K$ but $U \to -U$

Also $\langle c^\dagger_{i\downarrow} c_{j\downarrow}\rangle \to \langle \delta_{ij} -d^\dagger_{i\downarrow} d_{j\downarrow}\rangle$ 

In [639]:
o_up,o_dn = of,signs_v.reshape(N,1)* ov
g_up,g_dn = greens_1body(o_up),np.eye(N)-signs*greens_1body(o_dn)

U=3.5
# see 10.1103/PhysRevX.10.031016 for formula
np.sum(T*(g_up))+np.sum((g_dn)*T)+ U*(g_up.diagonal()@(g_dn.diagonal())) 


np.float64(-9.265465844048931)

Alternate way to write the above

In [640]:
o_up,o_dn = of,signs_v.reshape(N,1)* ov
g_up,g_dn = greens_1body(o_up),greens_1body(o_dn)

U=3.5
# see 10.1103/PhysRevX.10.031016 for formula
np.sum(T*(g_up))+np.sum((g_dn)*T)+ -U*(g_up.diagonal()@(g_dn.diagonal())) + U*np.sum(g_up.diagonal())


np.float64(-9.265465844048933)

These are all partial transforms that work out

#### Random Orbitals (RHF)

In [641]:
rng = np.random.default_rng(42)
ot = rng.random(of.shape)
ot = np.linalg.qr(ot)[0]
gt = greens_1body(ot)

g_up,g_dn = greens_1body(gt),greens_1body(gt)
print(np.sum(T*g_up)+np.sum((g_dn)*T)+ U*(g_up.diagonal()@g_dn.diagonal()) )

# virtual orbitals
gtv = signs_v.reshape(N,1)*sl.null_space(gt)

g_up,g_dn = np.eye(N)-signs*greens_1body(gtv),np.eye(N)-signs*greens_1body(gtv)
print(np.sum(T*(g_up))+np.sum((g_dn)*T)+ U*(g_up.diagonal()@(g_dn.diagonal())) )

4.80693418845771
4.806934188457705


### Rank Deficient (doped)

In [642]:
rng = np.random.default_rng(42)
Trand = rng.random((N,N))
Trand += Trand.T.conj()
Trand -= np.diag(Trand.diagonal())
lr,vecs_r = np.linalg.eigh(Trand)

ot = vecs_r[:,:3]
gt = greens_1body(ot)

g_up,g_dn = greens_1body(gt),greens_1body(gt)
print(np.sum(T*g_up)+np.sum((g_dn)*T)+ U*(g_up.diagonal()@g_dn.diagonal()) )


gtv = signs_v.reshape(N,1)*sl.null_space(gt)

g_up,g_dn = np.eye(N)-signs*greens_1body(gtv),np.eye(N)-signs*greens_1body(gtv)
print(np.sum(T*(g_up))+np.sum((g_dn)*T)+ U*(g_up.diagonal()@(g_dn.diagonal())) )

3.892250945395796
3.8922509453957974


Why did this work? Because we used the null space which filled more orbitals than we should have. If we repeat this with the proper filling setup

In [643]:


gtv = signs_v.reshape(N,1)*vecs_r[:,-3:]

g_up,g_dn = np.eye(N)-signs*greens_1body(gtv),np.eye(N)-signs*greens_1body(gtv)
print(np.sum(T*(g_up))+np.sum((g_dn)*T)+ U*(g_up.diagonal()@(g_dn.diagonal())) )

43.5094906548905


And now we see it properly break

## Charge Fields (chemical potential)

Adding a charge field $H_B = \sum_i h_i n_i = \sum_i h_i(n_{i\uparrow}+n_{i\downarrow})$ 

In [644]:
rng = np.random.default_rng(42)
# h_i = rng.random(N)
h_i = np.ones(N)*-U/2
Tup = T+np.diag(h_i)
Tdn = Tup

lamsB,vecsB = np.linalg.eigh(Tup)
# filled orbitals, and unfilled (virtual) orbitals
ofB_up,ovB_up = vecsB[:,:8], vecsB[:,8:]


lamsB2,vecsB2 = np.linalg.eigh(Tdn)
# filled orbitals, and unfilled (virtual) orbitals
ofB_dn,ovB_dn = vecsB2[:,:8], vecsB2[:,8:]

In [645]:
o_up,o_dn = ofB_up,ofB_dn
g_up,g_dn = greens_1body(o_up),greens_1body(o_dn)

U=3.5
np.sum((Tup)*g_up)+np.sum((g_dn)*Tdn)+ U*(g_up.diagonal()@g_dn.diagonal())

np.float64(-37.12489193478544)

the charge is transformed by 
$$
n_i \to 1-m_{i\uparrow} + 1-m_{i\downarrow} = 2-n_i
$$

In [646]:
o_up,o_dn = signs_v.reshape(N,1)* ovB_up,signs_v.reshape(N,1)* ovB_dn
g_up,g_dn =np.eye(N)-greens_1body(o_up)*signs,np.eye(N)-greens_1body(o_dn)*signs

U=3.5
Tup2,Tdn2 = T+np.diag(-h_i), T+np.diag(-h_i)

np.sum((Tup2)*(g_up))+np.sum((g_dn)*(Tdn2))+ U*(g_up.diagonal()@(g_dn.diagonal())) + 2*np.sum(h_i)


np.float64(-37.12489193478544)

### Different Chemical Potential

In [653]:
rng = np.random.default_rng(42)
h_i = -rng.random(N)
# h_i = np.ones(N)*-0.1
Tup = T+np.diag(h_i)
Tdn = Tup

lamsB,vecsB = np.linalg.eigh(Tup)
# filled orbitals, and unfilled (virtual) orbitals
ofB_up,ovB_up = vecsB[:,:8], vecsB[:,8:]


lamsB2,vecsB2 = np.linalg.eigh(Tdn)
# filled orbitals, and unfilled (virtual) orbitals
ofB_dn,ovB_dn = vecsB2[:,:8], vecsB2[:,8:]

In [654]:
o_up,o_dn = ofB_up,ofB_dn
g_up,g_dn = greens_1body(o_up),greens_1body(o_dn)

U=3.5
np.sum((Tup)*g_up)+np.sum((g_dn)*Tdn)+ U*(g_up.diagonal()@g_dn.diagonal())

np.float64(-18.70601738095342)

In [655]:
o_up,o_dn = signs_v.reshape(N,1)* ovB_up,signs_v.reshape(N,1)* ovB_dn
g_up,g_dn =np.eye(N)-greens_1body(o_up)*signs,np.eye(N)-greens_1body(o_dn)*signs

U=3.5
Tup2,Tdn2 = T+np.diag(-h_i), T+np.diag(-h_i)

np.sum((Tup2)*(g_up))+np.sum((g_dn)*(Tdn2))+ U*(g_up.diagonal()@(g_dn.diagonal())) + 2*np.sum(h_i)


np.float64(-16.61937267417322)

## B Field (collinear)

Adding a magnetic field $H_B = \sum_i h_i S_z = \sum_i h_i \frac{1}{2}(n_{i\uparrow}-n_{i\downarrow})$ 

In [678]:
rng = np.random.default_rng(42)
h_i = rng.random(N)*0.1/2
# h_i = np.ones(N)* 0.1
Tup,Tdn = T+np.diag(h_i), T-np.diag(h_i)

lamsB,vecsB = np.linalg.eigh(Tup)
# filled orbitals, and unfilled (virtual) orbitals
ofB_up,ovB_up = vecsB[:,:8], vecsB[:,8:]


lamsB2,vecsB2 = np.linalg.eigh(Tdn)
# filled orbitals, and unfilled (virtual) orbitals
ofB_dn,ovB_dn = vecsB2[:,:8], vecsB2[:,8:]

In [679]:
o_up,o_dn = ofB_up,ofB_dn
g_up,g_dn = greens_1body(o_up),greens_1body(o_dn)

U=3.5
np.sum((Tup)*g_up)+np.sum((g_dn)*Tdn)+ U*(g_up.diagonal()@g_dn.diagonal())

np.float64(-11.39629313609467)

This seems to accidentally work

In [680]:
o_up,o_dn = signs_v.reshape(N,1)* ovB_up,signs_v.reshape(N,1)* ovB_dn
g_up,g_dn =np.eye(N)-greens_1body(o_up)*signs,np.eye(N)-greens_1body(o_dn)*signs

U=3.5
np.sum(Tup*(g_up))+np.sum((g_dn)*(Tdn))+ U*(g_up.diagonal()@(g_dn.diagonal())) 


np.float64(-11.396293136094672)

But 
$$
S_z \to (1-m_{i\uparrow} - (1-m_{i\downarrow})) = -S_z
$$

In [681]:
o_up,o_dn = signs_v.reshape(N,1)* ovB_up,signs_v.reshape(N,1)* ovB_dn
g_up,g_dn =np.eye(N)-greens_1body(o_up)*signs,np.eye(N)-greens_1body(o_dn)*signs

U=3.5
Tup2,Tdn2 = T-np.diag(h_i), T+np.diag(h_i)

np.sum((Tup2)*(g_up))+np.sum((g_dn)*(Tdn2))+ U*(g_up.diagonal()@(g_dn.diagonal())) 


np.float64(-11.33575040704775)

The Partial transform should be then

$$
S_z \to (n_{i\uparrow} - (1-m_{i\downarrow}))
$$

In [682]:
signs_v = (-1)**(np.array([np.sum(s.coord)  for s in lattice.sites]))
o_up,o_dn = ofB_up,signs_v.reshape(N,1)* ovB_dn
signs = np.outer(signs_v,signs_v)

g_up,g_dn = greens_1body(o_up),np.eye(N)-signs*greens_1body(o_dn)

U=3.5
Tup2,Tdn2 = T+np.diag(h_i), T+np.diag(h_i)

np.sum((Tup2)*(g_up))+np.sum((g_dn)*(Tdn2))+ U*(g_up.diagonal()@(g_dn.diagonal()))+ -np.sum(h_i)


np.float64(-11.366021771571209)